# Scripts que devuelve las diferencias entre los ficheros

Importamos

In [62]:
"""
    This is the main function.
"""
print('Importing libraries...')
import json
import os
import pandas as pd
from difflib import SequenceMatcher
import jellyfish
from geopy.geocoders import Nominatim
geolocator = Nominatim()
from geopy.distance import great_circle
import folium
from folium.plugins import MarkerCluster
from convertbng.util import convert_bng, convert_lonlat, convert_etrs89_to_lonlat
import overpass
import sys

Importing libraries...


In [63]:
# Functions to work
print('Defining functions...')
def get_id(x):
    try:
        for i in range(len(ids)):
            point = [lat[i],lon[i]]
            if getDistance_meters(x,point) <results[fichero]['distance_SAME']:
                ayu[i] = x
                return ids[i]
        return 'No match'
    except:
        return 'No match'

def get_close(x):
    try:
        for i in range(len(ids)):
            point = [lat[i],lon[i]]
            if getDistance_meters(x,point) <results[fichero]['distance_CLOSE']:
                ayu[i] = x
                return ids[i]
        return 'No match'
    except:
        return 'No match'

def get_lat(x):
    try:
        ind = ids.index(x)
        return float(lat[ind])
    except:
        return None

def get_lon(x):
    try:
        ind = ids.index(x)
        return float(lon[ind])
    except:
        return None

def get_name(x):
    try:
        ind = ids.index(x)
        return name[ind]
    except:
        return None

def get_score(x):
    return getDistance_meters([x.lat_OSM, x.lon_OSM],[x.LATITUD, x.LONGITUD])

def getDistance_meters(x,y):
    try:
        return great_circle(x, y).kilometers*1000.0
    except:
        return 0

Defining functions...


In [64]:
first_arg = sys.argv[1]

In [65]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/data/')
os.listdir()

['20170614-BANCOS.csv',
 '20170614-PAPELERAS.csv',
 '20171110-InventarioFuentes-edit.csv',
 '20171110-InventarioFuentes-missing_AYU.csv',
 '20171110-InventarioFuentes.csv',
 'anexo i monumentos.xls',
 'doctordata.py',
 'indice.csv']

In [66]:
AYU_fichero = '20171110-InventarioFuentes.csv'
# Function to pass argument in terminal
try:
    if (first_arg != '-f'):
        AYU_fichero = first_arg
except:
    pass

In [67]:
#Center coordinates
SF_COORDINATES = (40.4168, -3.7038)

# for speed purposes
MAX_RECORDS = 10



print('Opening files and ensuring naming convention...')
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/data/')
indice = pd.read_csv('indice.csv',index_col=0,encoding='latin-1',delimiter=';')

results = {}
for key, df_gb in indice.groupby(indice.index):
    results[str(key)] = df_gb.to_dict('list')

for key in results.keys():
    results[key]['SEARCH'] = results[key]['SEARCH'][0][1:-1]
    results[key]['distance_CLOSE'] = results[key]['distance_CLOSE'][0]
    results[key]['distance_SAME'] = results[key]['distance_SAME'][0]
    results[key]['palabra'] = results[key]['palabra'][0]


fichero = []
for key in results.keys():
    if AYU_fichero.split('-')[1][:-4] == results[key]['palabra']:
        fichero = key

search = 'area["name"="Madrid"];node(area)['+results[fichero]['SEARCH']+'];'

api = overpass.API()
OSM = api.Get(search)

#OSM['features'][0]


# Create searching arrays

ids = [OSM['features'][i]['id'] for i in range(len(OSM['features']))]
lat = [OSM['features'][i]['geometry']['coordinates'][1] for i in range(len(OSM['features']))]
lon = [OSM['features'][i]['geometry']['coordinates'][0] for i in range(len(OSM['features']))]
osm = list(zip(ids,lat,lon))
#name = [OSM['features'][i]['properties']['name'] for i in range(len(OSM['features']))]
ayu = ids[:]
print('Total OSM entities found: ',str(len(ayu)))

# Add ayuntamiento data
#results[fichero]['DOCTORDATA'][1:]
AYU = pd.read_csv(AYU_fichero, encoding='latin-1', delimiter=';',index_col=0)
AYU.columns = [x.upper() for x in AYU.columns.values]

print('Total MAD entities found: ', str(AYU.count().max()))
if AYU_fichero == '20171110_InventarioFuentes.csv':
    AYU.columns = results[fichero]['DOCTORDATA'][1:]

try:
    AYU = AYU[AYU.LATITUD != 'Error']
    AYU = AYU[AYU.LONGITUD != 'Error']
except:
    print('No errors!')

AYU.LATITUD = AYU.LATITUD.astype('float32')
AYU.LONGITUD = AYU.LONGITUD.astype('float32')
AYU = AYU.sort_values('LATITUD')
#AYU['X_ETRS89'] = AYU['X_ETRS89'].apply(lambda x: float(x.replace(',','.')))
#AYU['Y_ETRS89'] = AYU['Y_ETRS89'].apply(lambda x: float(x.replace(',','.')))
AYU['coord'] = list(zip(AYU.LATITUD, AYU.LONGITUD))
#AYU = AYU[:200]

Opening files and ensuring naming convention...
Total OSM entities found:  1639
Total MAD entities found:  1605


In [68]:
columns = ['id_OSM','lat_OSM','lon_OSM']
osm_df = pd.DataFrame(osm, columns=columns)
osm_df['coord_OSM'] = osm_df.apply(lambda x: (x.lat_OSM,x.lon_OSM), axis=1)
AYU_OK = []
AYU_EDIT = []
AYU_NO = []

In [69]:
for index,row in AYU.iterrows():
    osm_df['distance'] = osm_df.apply(lambda x: getDistance_meters(x.coord_OSM,row.coord),axis=1)
    osm_df = osm_df.sort_values('distance')
    
    if osm_df['distance'].head(1).values < 10:
        AYU_OK.append([index,osm_df.head(1)['id_OSM'].values[0], osm_df.head(1)['distance'].values[0]])
    elif osm_df['distance'].head(1).values < 50:
        AYU_EDIT.append([index,osm_df.head(1)['id_OSM'].values[0], osm_df.head(1)['distance'].values[0]])
    else:
        AYU_NO.append([index,osm_df.head(1)['id_OSM'].values[0], osm_df.head(1)['distance'].values[0]])

In [97]:
columns = ['CODIGO', 'id_OSM', 'distance']
ayu_ok_df = pd.DataFrame(AYU_OK, columns=columns)
ayu_edit_df = pd.DataFrame(AYU_EDIT, columns=columns)

In [98]:
osm_df = osm_df.sort_values('id_OSM')
ayu_ok_df = ayu_ok_df.sort_values('id_OSM')
ayu_edit_df = ayu_edit_df.sort_values('id_OSM')

In [99]:
ayu_missing_df = osm_df[~osm_df.id_OSM.isin(list(ayu_ok_df.id_OSM.values))]

In [100]:
ayu_edit_df['lat_OSM'] = ayu_edit_df['id_OSM'].apply(lambda x: get_lat(x))
ayu_edit_df['lon_OSM'] = ayu_edit_df['id_OSM'].apply(lambda x: get_lon(x))
ayu_edit_df['lat_OSM'] = ayu_edit_df['lat_OSM'].apply(lambda x: round(float(x),7))
ayu_edit_df['lon_OSM'] = ayu_edit_df['lon_OSM'].apply(lambda x: round(float(x),7))
ayu_edit_df['coord_OSM'] = list(zip(ayu_edit_df.lat_OSM, ayu_edit_df.lon_OSM))

ayu_missing_df['lat_OSM'] = ayu_missing_df['id_OSM'].apply(lambda x: get_lat(x))
ayu_missing_df['lon_OSM'] = ayu_missing_df['id_OSM'].apply(lambda x: get_lon(x))
ayu_missing_df['lat_OSM'] = ayu_missing_df['lat_OSM'].apply(lambda x: round(float(x),7))
ayu_missing_df['lon_OSM'] = ayu_missing_df['lon_OSM'].apply(lambda x: round(float(x),7))
ayu_missing_df['coord_OSM'] = list(zip(ayu_missing_df.lat_OSM, ayu_missing_df.lon_OSM))

/Users/raimundoabrillopez/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/raimundoabrillopez/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/raimundoabrillopez/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

In [102]:
ayu_edit_df= ayu_edit_df.set_index('CODIGO')
ayu_missing_df= ayu_missing_df.set_index('id_OSM')

In [106]:
ayu_edit_df = pd.join([ayu_edit_df, AYU], axis=1, join='inner')

AttributeError: module 'pandas' has no attribute 'join'

In [ ]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/csv/')
print('Saving to files...')
ayu_edit_df.to_csv(AYU_fichero[:-4]+'-edit.csv')
#AYU_NO.to_csv(AYU_fichero[:-4]+'-missing_OSM.csv')
ayu_missing_df.to_csv(AYU_fichero[:-4]+'-missing_AYU.csv')

In [24]:
print('Get id matching....')
AYU.id_OSM = AYU.coord.apply(lambda x: get_id(x))
AYU_NO = AYU[AYU.id_OSM == 'No match']
AYU_OK = AYU[AYU.id_OSM != 'No match']
AYU_CLOSE = AYU_NO.copy()

Get id matching....


In [7]:
print('Get close entities...')
AYU_CLOSE.id_OSM = AYU_NO.coord.apply(lambda x: get_close(x))
AYU_NO = AYU_CLOSE[AYU_CLOSE.id_OSM == 'No match']
AYU_CLOSE = AYU_CLOSE[AYU_CLOSE.id_OSM != 'No match']

AYU = pd.concat([AYU_OK, AYU_CLOSE, AYU_NO])

print('Geolocating from OSM...')
AYU.lat_OSM = AYU.id_OSM.apply(lambda x: get_lat(x))
AYU.lon_OSM = AYU.id_OSM.apply(lambda x: get_lon(x))
AYU['coord'] = list(zip(AYU.lat_OSM, AYU.lon_OSM))
AYU.score_OSM = AYU.apply(lambda x: get_score(x), axis=1)
AYU['coord_OSM'] = list(zip(AYU.lat_OSM, AYU.lon_OSM))

try:
    AYU_CLOSE.lat_OSM = AYU_CLOSE.id_OSM.apply(lambda x: get_lat(x))
    AYU_CLOSE.lon_OSM = AYU_CLOSE.id_OSM.apply(lambda x: get_lon(x))
    AYU_CLOSE['coord'] = list(zip(AYU_CLOSE.lat_OSM, AYU_CLOSE.lon_OSM))
    AYU_CLOSE.score_OSM = AYU_CLOSE.apply(lambda x: get_score(x), axis=1)
    AYU_CLOSE['coord_OSM'] = list(zip(AYU_CLOSE.lat_OSM, AYU_CLOSE.lon_OSM))
except:
    print('No close')

AYU_NO.lat_OSM = AYU_NO.id_OSM.apply(lambda x: get_lat(x))
AYU_NO.lon_OSM = AYU_NO.id_OSM.apply(lambda x: get_lon(x))
AYU_NO['coord'] = list(zip(AYU_NO.lat_OSM, AYU_NO.lon_OSM))
AYU_NO.score_OSM = AYU_NO.apply(lambda x: get_score(x), axis=1)
AYU_NO['coord_OSM'] = list(zip(AYU_NO.lat_OSM, AYU_NO.lon_OSM))

AYU = AYU.sort_values('score_OSM')
#AYU.name_OSM = AYU.id_OSM.apply(lambda x: get_name(x))

print('Creating missing Dataframe...')
AYU_MISSING = []
for i in range(len(ids)):
    AYU_MISSING.append((ids[i],lat[i],lon[i]))

AYU_MISSING = pd.DataFrame(AYU_MISSING)
AYU_MISSING.columns = ['id_OSM','lat_OSM','lon_OSM']

AYU_MISSING = AYU_MISSING.set_index('id_OSM')
AYU_MISSING['coord'] = list(zip(AYU_MISSING.lat_OSM.values, AYU_MISSING.lon_OSM.values))

os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/csv/')
print('Saving to files...')
AYU_CLOSE.to_csv(AYU_fichero[:-4]+'-edit.csv')
AYU_NO.to_csv(AYU_fichero[:-4]+'-missing_OSM.csv')
AYU_MISSING.to_csv(AYU_fichero[:-4]+'-missing_AYU.csv')


Opening files and ensuring naming convention...
Total OSM entities found:  1639


FileNotFoundError: File b'20171110_InventarioFuentes.csv' does not exist

## Close to edit

In [33]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/web/')

In [43]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11,max_zoom=20, control_scale = True, prefer_canvas=True)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_CLOSE.id_OSM.values)):
    folium.Marker(icon=folium.Icon(color='blue'),popup=folium.Popup(str(AYU_CLOSE.index.values[i]), parse_html=True),
        location = (AYU_CLOSE.LATITUD.values[i],AYU_CLOSE.LONGITUD.values[i])).add_to(marker_cluster)
map.save(AYU_fichero[:-4]+'-edit.html')

## Missing in OSM

In [35]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11,max_zoom=18, control_scale = True, prefer_canvas=True)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_NO.id_OSM.values[:100])):
    folium.Marker(icon=folium.Icon(color='blue'),popup=folium.Popup(AYU_NO.index.values[i], parse_html=True),
        location = (AYU_NO.LATITUD.values[i],AYU_NO.LONGITUD.values[i])).add_to(marker_cluster)
map.save(AYU_fichero[:-4]+'-missing_OSM.html')

## Missing in Ayu website

In [36]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11,max_zoom=20, control_scale = True, prefer_canvas=True)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_MISSING.index.values)):
    folium.Marker(icon=folium.Icon(color='green'),popup=folium.Popup(AYU_MISSING.index.values[i], parse_html=True),
        location = (AYU_MISSING.lat_OSM.values[i],AYU_MISSING.lon_OSM.values[i])).add_to(marker_cluster)
map.save(AYU_fichero[:-4]+'-missing_AYU.html')